In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import graphviz 
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import LabelEncoder



base_dados = pd.read_excel('./dataset.xlsx')


column_drops = ['Patient ID', 
                'Patient addmited to intensive care unit (1=yes, 0=no)',
                'Patient addmited to regular ward (1=yes, 0=no)',
                'Patient addmited to semi-intensive unit (1=yes, 0=no)',
                ]


data_clean = base_dados.drop(columns=column_drops)


# Imprimindo informações iniciais sobre o dataset
print("Dataset All: {} linhas | {} colunas".format(data_clean.shape[0], data_clean.shape[1]))

# Calculando a porcentagem máxima de valores nulos permitida
porcentagem_maxima_nulos = data_clean.isna().mean().mean()

# Eliminando as colunas com porcentagem de valores nulos maior ou igual à porcentagem máxima permitida: 95%
data_clean.dropna(thresh=len(data_clean)*(1-porcentagem_maxima_nulos), axis=1, inplace=True)

print("Dataset Clean: {} linhas | {} colunas".format(data_clean.shape[0], data_clean.shape[1]))


# identifica as colunas que são do tipo 'object'

label_encoder = LabelEncoder()

cols_object = data_clean.select_dtypes(include=['object']).columns

# aplica a transformação LabelEncoder nas colunas identificadas
data_clean[cols_object] = data_clean[cols_object].apply(lambda col: label_encoder.fit_transform(col.astype(str)))

# preenche os valores nulos com a média de cada coluna
data_clean = data_clean.fillna(data_clean.mean())

# calcula a correlação com a coluna "SARS-Cov-2 exam result"
coluna_result = "SARS-Cov-2 exam result"
correlacao_01 =  data_clean.corrwith(data_clean[coluna_result], numeric_only=True)


# exibe a quantidade de colunas transformadas
print(f'Colunas que foram transformadas: {len(cols_object)}')


# Selecionar as 10 colunas com maior correlação com o resultado do teste COVID-19
m_cols = correlacao_01.abs().nlargest(10).index[1:]
